In [1]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data_cd/redux'

In [2]:
#Allow relative imports to directories above lesson1/
sys.path.insert(1, os.path.join(sys.path[0], '..'))

#import modules
from utils import *
from resnet50 import Resnet50

#Instantiate plotting tool
#In Jupyter notebooks, you will need to run this command before doing any plotting
%matplotlib inline
print ("Library Loaded")

Library Loaded


Using Theano backend.


In [3]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/' #'/sample/'
test_path = DATA_HOME_DIR + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

/home/ubuntu/Notebook/fastai/data_cd/redux


In [7]:
resnet=Resnet50(include_top=False).model

/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


94609408/94652608 [============================>.] - ETA: 0s

In [6]:
#Set constants. You can experiment with no_of_epochs to improve the model
batch_size=64
no_of_epochs=20

In [8]:
resnet.output_shape[1:]

(2048, 7, 7)

In [9]:
#Finetune the model
batches = get_batches(train_path, batch_size=batch_size)
val_batches = get_batches(valid_path, batch_size=batch_size*2)

(val_classes,trn_classes,val_labels,trn_labels,val_filenames,
filenames,test_filenames)=get_classes(path)

Found 22979 images belonging to 2 classes.
Found 1999 images belonging to 2 classes.
Found 22979 images belonging to 2 classes.
Found 1999 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [10]:
val_features=resnet.predict_generator(val_batches,val_batches.nb_sample)

In [12]:
import bcolz
save_array(results_path+"val_resnet_conv.bc",val_features)

In [13]:
trn_features=resnet.predict_generator(batches,batches.nb_sample)

In [17]:
# save_array(results_path+"trn_resnet_conv.bc",trn_features)

In [20]:
def get_fc_layers(p):
    return [
        BatchNormalization(axis=1,input_shape=resnet.output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(1024,activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(1024,activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(2,activation='softmax')
        
    ]

In [21]:
model=Sequential(get_fc_layers(0.5))

In [22]:
model.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [25]:
model.fit(trn_features,trn_labels,nb_epoch=2,batch_size=batch_size,validation_data=(val_features,val_labels))

INFO (theano.gof.compilelock): Refreshing lock /home/ubuntu/.theano/compiledir_Linux-3.13--generic-x86_64-with-debian-jessie-sid-x86_64-3.5.2-64/lock_dir/lock
Problem occurred during compilation with the command line below:
/usr/bin/g++ -shared -g -O3 -fno-math-errno -Wno-unused-label -Wno-unused-variable -Wno-write-strings -march=core-avx2 -mcx16 -msahf -mmovbe -maes -mpclmul -mpopcnt -mabm -mno-lwp -mfma -mno-fma4 -mno-xop -mbmi -mbmi2 -mno-tbm -mavx -mavx2 -msse4.2 -msse4.1 -mlzcnt -mno-rtm -mno-hle -mrdrnd -mf16c -mfsgsbase -mno-rdseed -mno-prfchw -mno-adx -mfxsr -mxsave -mxsaveopt --param l1-cache-size=32 --param l1-cache-line-size=64 --param l2-cache-size=25600 -mtune=generic -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION -m64 -fPIC -I/home/ubuntu/anaconda3/lib/python3.5/site-packages/numpy/core/include -I/home/ubuntu/anaconda3/include/python3.5m -I/home/ubuntu/anaconda3/lib/python3.5/site-packages/theano/gof -L/home/ubuntu/anaconda3/lib -fvisibility=hidden -o /home/ubuntu/.theano/c

OSError: [Errno 12] Cannot allocate memory